In [115]:
import pandas as pd
import requests as r
from bs4 import BeautifulSoup
import re
import time
from datetime import datetime

In [50]:
url='https://www.marmiton.org/recettes/recette_tacos-mexicains_34389.aspx'

In [51]:
r.get(url)

<Response [200]>

In [52]:
soup=BeautifulSoup(r.get(url).content)

In [ ]:
list_of selectors=['h1','div.recipe-author__author>span','div.recipe-infos-users__rate__stars>span','ul.mrtn-tags-list>li','span.recipe-infos-users__value','div.recipe-infos,div.recipe-infos__timmings__detail','div.recipe-infos__timmings__cooking,div.recipe-infos__timmings__preparation']

In [53]:
#Recipe Name
soup.select('h1')[0].text

'Tacos mexicains'

In [54]:
#Author name
soup.select('div.recipe-author__author>span')[0].text.strip('\n ')

'noor_16706694'

In [76]:
#recipe note
soup.select('div.recipe-infos-users__rate__stars>span')[0].text
[i.text for i in soup.select('div.recipe-infos-users__rate__stars>span')]

['4.6/5']

In [56]:
#tags ('entrée, 'plat unique','plats mexicains)
#[i.text for i in soup.select('ul.mrtn-tags-list>li')]

['Entrée', 'Plat unique', 'Plats mexicains']

In [64]:
#Nb likes
soup.select('span.recipe-infos-users__value')[1].text

'15k'

In [57]:
#Nb of portions
[i.replace('\n',' ') for i in soup.select('div.recipe-infos,div.recipe-infos__timmings__detail')[0].text.split('\n\n') if len(i)>1]

['temps 55 min', ' personnes 4', ' très facile', 'bon marché']

In [71]:
#cooking time  only
[re.sub('\t|\n|\xa0','',i.text) for i in soup.select('div.recipe-infos__timmings__cooking')]


['Cuisson:25 min']

In [72]:
#Preparation time 
[re.sub('\t|\n|\xa0','',i.text) for i in soup.select('div.recipe-infos__timmings__preparation')]


['Préparation: 30 min']

In [66]:
#total time
soup.select('div.recipe-infos__timmings__total-time ')[0].text

'\nTemps Total : 55 min\t\t\t\t\t\t\t\t'

In [69]:
#nb people
soup.select('span.title-2.recipe-infos__quantity__value')[0].text

'4'

In [ ]:
#Ingredients
[i.text.strip().replace('\n','') for i in soup.select('li.recipe-ingredients__list__item')]

In [81]:
# average cost
[i.text.strip('\n') for i in soup.select('div.recipe-infos__budget')]
        

['bon marché']

In [85]:
# difficulty
[i.text.strip('\n') for i in soup.select('div.recipe-infos__level')]

['très facile']

In [ ]:
#Ustensiles
[re.sub('\n\n|\t*',' ',i.text) for i in soup.select('ul.recipe-utensils__list')]

In [ ]:
[i.text for i in soup.select('ul.recipe-utensils__list')]

In [75]:
# recipe
re.sub('\t|\n','',soup.select('ol.recipe-preparation__list')[0].text)
[i.text for i in soup.select('ol.recipe-preparation__list')]

["\n\n\t\t\tA la poêle, faire dorer l'oignon émincé dans un peu d'huile d'olive. \t\t\n\n\t\t\tRajouter la viande, assaisonner et laisser cuire 5 min. \t\t\n\n\t\t\tLaver les feuilles de laitue. \t\t\n\n\t\t\tCouper les tomates et le poivron en petits dés. \t\t\n\n\t\t\tIncorporer le tout à la poêlée avec le coulis de tomate, et poursuivre la cuisson pendant 5 min. \t\t\n\n\t\t\tEgoutter les haricots rouges et les ajouter 2 min avant la fin de cuisson. \t\t\n\n\t\t\tHors du feu, ajuster l'assaisonnement et saupoudrer généreusement de cumin; on peut aussi rajouter quelques gouttes de Tabasco. \t\t\n\n\t\t\tGarnir les tortillas de préparation et les refermer en les roulant comme des crêpes. Disposer 1 feuille de laitue sur chaque tacos avant de servir.\t\t\n"]

In [ ]:
url1='https://www.marmiton.org/recettes/recherche.aspx?type=all&aqt=&start=50&page=4'

In [ ]:
soup1=BeautifulSoup(r.get(url1).content)

In [ ]:
[i['href'] for i in soup1.select('a.recipe-card-link')]


# Code aggregation

In [ ]:
# 14 recipes per page with page from no figure then 2 to 833
#https://www.marmiton.org/recettes/recherche.aspx?type=all&aqt=&start=10004&page=833

In [130]:
#liste of links for the pages - start with 5pages to test before launching 833 pages...
# pb le site change de pattern de pagination 
def List_urls(NbPage):
    urls=[]
    for i in range(NbPage):
        url=f'https://www.marmiton.org/recettes/recherche.aspx?type=all&aqt=&start={4+14*i}&page={i+1}'
        urls.append(url)
        i+=1
    return urls

page_urls=List_urls(2)    
print(page_urls)

['https://www.marmiton.org/recettes/recherche.aspx?type=all&aqt=&start=4&page=1', 'https://www.marmiton.org/recettes/recherche.aspx?type=all&aqt=&start=18&page=2']


In [131]:
#For one given page look for the list of recipes (links)
#14 recipes per pages (or 12 paging changing?) time sleep =0,5sec every requests)
def find_recipe_card_link(page_urls):
    recipes_of_all_pages=[]
    for url in page_urls:
        soup=BeautifulSoup(r.get(url).content)
        # list of recipes url which are in the page
        recipes_of_all_pages.extend([f'https://www.marmiton.org'+i['href'] for i in soup.select('a.recipe-card-link')])
        time.sleep(0.5)
    return recipes_of_all_pages
        
recipes_of_all_pages=find_recipe_card_link(page_urls) 
recipes_of_all_pages

['https://www.marmiton.org/recettes/recette_original-american-cookies-de-mike_39907.aspx',
 'https://www.marmiton.org/recettes/recette_blanquette-de-veau-facile_19219.aspx',
 'https://www.marmiton.org/recettes/recette_le-crumble-aux-pommes-du-chat-qui-tousse_10534.aspx',
 'https://www.marmiton.org/recettes/recette_gateau-au-yaourt_12719.aspx',
 'https://www.marmiton.org/recettes/recette_veritable-moelleux-au-chocolat_12825.aspx',
 'https://www.marmiton.org/recettes/recette_les-timbales-de-jeanne-saumon-a-la-mousse-de-courgettes-au-micro-ondes_21864.aspx',
 'https://www.marmiton.org/recettes/recette_gateau-aux-pommes-facile_13493.aspx',
 'https://www.marmiton.org/recettes/recette_meringue-pour-les-nuls_36610.aspx',
 'https://www.marmiton.org/recettes/recette_pancakes_15299.aspx',
 'https://www.marmiton.org/recettes/recette_cookie-cup-glace-vanille-et-pepites-de-chocolat_372241.aspx',
 'https://www.marmiton.org/recettes/recette_filet-mignon-en-croute_14407.aspx',
 'https://www.marmiton.o

In [109]:
def Collect_data(recipes_of_page):
    
    
    #recipe=dict() #dictionnary , 1 row of the final dataframe (Recipe name: [list of info])  recipe={'':[]}
    list_of_recipes=[]
    recipe=[]    
    for url in recipes_of_page:
        
       
        soup=BeautifulSoup(r.get(url).content)
        
        
        # Add Recipe Name
        
        name=recipe.append([i.text for i in soup.select('h1')])
      
            
        # add author name
        recipe.append([i.text.strip('\n ') for i in soup.select('div.recipe-author__author>span')]) 
        
        #add note
        recipe.append([i.text for i in soup.select('div.recipe-infos-users__rate__stars>span')]) 
        
        #add tags
        recipe.append([i.text for i in soup.select('ul.mrtn-tags-list>li.mrtn-tag>a')])
        
        # nb likes
        try:
            recipe.append(soup.select('span.recipe-infos-users__value')[1].text)
        except IndexError:
            recipe.append()
        
        #nb portions
        recipe.append([i.text.strip('\n') for i in soup.select('span.title-2.recipe-infos__quantity__value')])
        
        # difficulty
        recipe.append([i.text.strip('\n') for i in soup.select('div.recipe-infos__level')])
        
        #add total cooking time
        recipe.append([i.text.strip('\n') for i in soup.select('div.recipe-infos__timmings__total-time ')])
        
        # preparation time
        recipe.append([i.text.strip('\n') for i in soup.select('div.recipe-infos__timmings__preparation')])
            
        # cooking only time
        recipe.append([i.text.strip('\n') for i in soup.select('div.recipe-infos__timmings__cooking')])

        # average cost
        recipe.append([i.text.strip('\n') for i in soup.select('div.recipe-infos__budget')])
        
        # add url
        recipe.append(url)
        
        #ingredients
        recipe.append([i.text.strip('\n') for i in soup.select('li.recipe-ingredients__list__item')])
        
        #recipe (text)
        recipe.append([i.text.strip('\n') for i in soup.select('ol.recipe-preparation__list')])
        
        #check
        print(recipe)
        
        list_of_recipes.append(recipe)
        recipe=[]
    return list_of_recipes

#Collect_data(recipes_of_all_pages)

In [132]:
# new method
        
def Scrap_data(url):   
    
        
    soup=BeautifulSoup(r.get(url).content)
    
    # Recipe Name
    name=[i.text for i in soup.select('h1')]
       
    # Author name
    author_name=[i.text.strip('\n ') for i in soup.select('div.recipe-author__author>span')]
        
    #Rating
    rating= [i.text for i in soup.select('div.recipe-infos-users__rate__stars>span')]
        
    #Tags
    tags=[i.text for i in soup.select('ul.mrtn-tags-list>li.mrtn-tag>a')]
        
    # nb likes
    try:
        nblikes=soup.select('span.recipe-infos-users__value')[1].text
    except IndexError:
        nblikes=''
        
    #nb portions
    nbportions=[i.text.strip('\n') for i in soup.select('span.title-2.recipe-infos__quantity__value')]
        
    # difficulty
    difficulty=[i.text.strip('\n') for i in soup.select('div.recipe-infos__level')]
        
    #Total cooking time
    totaltime=[i.text.strip('\n') for i in soup.select('div.recipe-infos__timmings__total-time ')]
        
    # preparation time
    preparationtime=[i.text.strip('\n') for i in soup.select('div.recipe-infos__timmings__preparation')]
            
    # cooking only time
    cookingtime=[i.text.strip('\n') for i in soup.select('div.recipe-infos__timmings__cooking')]

    # cost range
    costrange=[i.text.strip('\n') for i in soup.select('div.recipe-infos__budget')]
        
    # add url
    link=url
        
    #list of ingredients
    loi=[i.text.strip('\n') for i in soup.select('li.recipe-ingredients__list__item')]
        
    #recipe (text)
    recipedescr=[i.text.strip('\n') for i in soup.select('ol.recipe-preparation__list')]
        
    dct={'Recipe_name':name,\
         'Author_name':author_name,\
         'Rating':rating,\
         'Tags':tags,\
         'Nb_likes':nblikes,\
         'Nb_portions':nbportions,\
         'Difficulty':difficulty,\
         'Total_cooking_time':totaltime,\
         'Preparation_time':preparationtime,\
         'Cooking_time':cookingtime,\
         'Cost_range':costrange,\
         'List_of_ingredients':loi,\
         'Recipe_description':recipedescr,\
         'Link':link}
    return dct




In [141]:

# Get all recipes for all pages
data=[]
column_names=['Recipe_name',\
         'Author_name',\
         'Rating',\
         'Tags',\
         'Nb_likes',\
         'Nb_portions',\
         'Difficulty',\
         'Total_cooking_time',\
         'Preparation_time',\
         'Cooking_time',\
         'Cost_range',\
         'List_of_ingredients',\
         'Recipe_description',\
         'Link']
df = pd.DataFrame(columns = column_names) # create an empty dataframe 
for nbscrap, url in enumerate(recipes_of_all_pages):
    #scrap info from recipes
    output=Scrap_data(url)
    data.append(output)
    if nbscrap%5==0:
        time.sleep(2)
        print(f'{nbscrap} reached')
    # save every n recipes n a file
    if nbscrap%10==0:
        df=pd.DataFrame(data)
        df.to_csv('/users/sandrine/Desktop/Prework/Projects_Ironhack/web_scraping/marmiton.csv')
df=pd.DataFrame(data)
df.to_csv('/users/sandrine/Desktop/Prework/Projects_Ironhack/web_scraping/marmiton.csv')

0 reached
5 reached
10 reached
15 reached
20 reached
25 reached


In [137]:
df

,Recipe_name,Author_name,Rating,Tags,Nb_likes,Nb_portions,Difficulty,Total_cooking_time,Preparation_time,Cooking_time,Cost_range,List_of_ingredients,Recipe_description,Link
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [120]:
df

,Recipe_name,Author_name,Rating,Tags,Nb_likes,Nb_portions,Difficulty,Total_cooking_time,Preparation_time,Cooking_time,Cost_range,List_of_ingredients,Recipe_description,Link
0,[Original American Cookies de Mike],[nathan_15813882],[4.7/5],"[Végétarien, Rapide, Petits gâteaux, Gâteau, C...",121k,[15],[très facile],[Temps Total : 15 min\t\t\t\t\t\t\t\t],[Préparation : 5 min\t\t\t\t\t\t\t\t\t\t],[Cuisson :\n\n\t\t\t\t\t\t\t\t\t\t\t\t10 min\t...,[bon marché],"[250\n\n\n\ng de farine , 90\n\n\n\ng de sucr...","[\t\t\tMélangez la farine, les sucres, le sel ...",https://www.marmiton.org/recettes/recette_orig...
1,[Blanquette de veau facile],[lilia1],[4.9/5],"[Viande en sauce, Viande, Blanquette, Je débute]",151k,[4],[très facile],[Temps Total : 2h15\t\t\t\t\t\t\t\t],[Préparation : 15 min\t\t\t\t\t\t\t\t\t\t],[Cuisson :\n\n\t\t\t\t\t\t\t\t\t\t\t\t2 h\t\t\...,[Coût moyen],"[1\n\n\n\nkg de blanquette de veau , 1\n\n\n\...",[\t\t\tFaire revenir la viande dans un peu de ...,https://www.marmiton.org/recettes/recette_blan...
2,[Le crumble aux pommes du Chat qui Tousse],[Anonyme],[4.8/5],"[Végétarien, Crumble, Crumble aux pommes, Les ...",104k,[6],[très facile],[Temps Total : 55 min\t\t\t\t\t\t\t\t],[Préparation : 25 min\t\t\t\t\t\t\t\t\t\t],[Cuisson :\n\n\t\t\t\t\t\t\t\t\t\t\t\t30 min\t...,[bon marché],"[6\n\n\n\n pommes (des Canada par exemple) , 1...",[\t\t\tPréchauffer le four à 210°C (thermostat...,https://www.marmiton.org/recettes/recette_le-c...
3,[Gâteau au yaourt],[Anonyme],[4.6/5],"[Végétarien, Gâteau, Gâteau au yaourt, Dessert...",120k,[4],[très facile],[Temps Total : 45 min\t\t\t\t\t\t\t\t],[Préparation : 15 min\t\t\t\t\t\t\t\t\t\t],[Cuisson :\n\n\t\t\t\t\t\t\t\t\t\t\t\t30 min\t...,[bon marché],"[1/2\n\n\n\npaquet de levure chimique , 1\n\n...",[\t\t\tPréchauffer le four à 180°C (thermostat...,https://www.marmiton.org/recettes/recette_gate...
4,[Véritable moelleux au chocolat],[Anonyme],[4.6/5],"[Végétarien, Gâteau, Gâteau au chocolat, Desse...",89k,[6],[très facile],[Temps Total : 40 min\t\t\t\t\t\t\t\t],[Préparation : 10 min\t\t\t\t\t\t\t\t\t\t],[Cuisson :\n\n\t\t\t\t\t\t\t\t\t\t\t\t30 min\t...,[bon marché],"[125\n\n\n\ng de farine , 125\n\n\n\ng de beu...",[\t\t\tPréchauffer le four à 180°C (thermostat...,https://www.marmiton.org/recettes/recette_veri...
5,[Les Timbales de Jeanne (saumon à la mousse de...,[sofia_15984013],[4.8/5],"[Sans gluten, Micro-ondes]",78k,[4],[très facile],[Temps Total : 18 min\t\t\t\t\t\t\t\t],[Préparation : 15 min\t\t\t\t\t\t\t\t\t\t],[Cuisson :\n\n\t\t\t\t\t\t\t\t\t\t\t\t3 min\t\...,[bon marché],"[4\n\n\n\ntranches de saumon fumé , 2\n\n\n\n...","[\t\t\tRincer, essorer et ciseler l'aneth et l...",https://www.marmiton.org/recettes/recette_les-...
6,[Gâteau aux pommes facile],[Anonyme],[4.7/5],"[Végétarien, Gâteau, Gâteau aux pommes, Desser...",89k,[1],[très facile],[Temps Total : 57 min\t\t\t\t\t\t\t\t],[Préparation : 12 min\t\t\t\t\t\t\t\t\t\t],[Cuisson :\n\n\t\t\t\t\t\t\t\t\t\t\t\t45 min\t...,[bon marché],"[120\n\n\n\ng de sucre semoule , 1\n\n\n\npaq...",[\t\t\tPréchauffer le four à 160°C (thermostat...,https://www.marmiton.org/recettes/recette_gate...
7,[Meringue pour les nuls],[julia_15932643],[4.8/5],"[Végétarien, Sans gluten, Petits gâteaux, Gâte...",73k,[20],[très facile],[Temps Total : 1h05\t\t\t\t\t\t\t\t],[Préparation : 5 min\t\t\t\t\t\t\t\t\t\t],[Cuisson :\n\n\t\t\t\t\t\t\t\t\t\t\t\t1 h\t\t\...,[bon marché],"[4\n\n\n\n blancs d'oeuf , 250\n\n\n\ng de su...",[\t\t\tQuelques trucs pour bien réussir ces me...,https://www.marmiton.org/recettes/recette_meri...
8,[Pancakes],[Anonyme],[4.7/5],"[Végétarien, Rapide, Crêpe, Pancake, Les class...",109k,[4],[très facile],[Temps Total : 14 min\t\t\t\t\t\t\t\t],[Préparation : 10 min\t\t\t\t\t\t\t\t\t\t],[Cuisson :\n\n\t\t\t\t\t\t\t\t\t\t\t\t4 min\t\...,[bon marché],"[250\n\n\n\ng de farine , 30\n\n\n\ng de sucr...","[\t\t\tFaire fondre le beurre, dans une casser...",https://www.marmiton.org/recettes/recette_panc...
9,[Cookie cup glace vanille et pépites d

In [ ]:
# Launch function terminal

In [ ]:
url_page='https://www.marmiton.org/recettes/recherche.aspx?type=all&aqt=&start=18&page=2'
soup=BeautifulSoup(r.get(url_page).content)
[f'https://www.marmiton.org'+i['href'] for i in soup.select('a.recipe-card-link')]

In [ ]:
https://www.marmiton.org/recettes/recette_chouquettes_30622.aspx

In [ ]:
#